# NDVI Analysis with Landsat 8 using OpenGeo (GEE Style)

This notebook demonstrates how to perform NDVI analysis using the `opengeo` package. The API is designed to feel like Google Earth Engine (GEE) but runs locally using `xarray`, `stackstac`, and `leafmap`.

## 1. Initialize OpenGeo

OpenGeo can be initialized with various STAC API endpoints. You can use direct URLs or convenient aliases.

In [1]:
import opengeo as og

# Examples of different STAC API initializations:

# 1. Element84 Earth Search (Sentinel-2, Landsat)
# og.Initialize("https://earth-search.aws.element84.com/v1")

# 2. Microsoft Planetary Computer (Recommended for Landsat 8 free access)
og.Initialize("MICROSOFT") # Alias for https://planetarycomputer.microsoft.com/api/stac/v1

# 3. NASA CMR STAC
# og.Initialize("NASA") # Alias for https://cmr.earthdata.nasa.gov/stac

# 4. USGS LandsatLook
# og.Initialize("USGS") # Alias for https://landsatlook.usgs.gov/stac-server

# 5. Google Earth Engine STAC
# og.Initialize("GEE") # Alias for https://earthengine-stac.googleapis.com/v1beta


C:\Users\pulak\anaconda3\envs\maps\Lib\site-packages\pyproj\network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)


OpenGeo initialized with STAC API: https://planetarycomputer.microsoft.com/api/stac/v1


## 2. Define Region of Interest (ROI)

Define a rectangular region over Bangalore.

In [2]:
roi = og.Geometry.Rectangle(77.55, 12.90, 77.65, 13.00)
print("ROI:", roi)

ROI: og.Geometry(POLYGON ((77.65 12.9, 77.65 13, 77.55 13, 77.55 12.9, 77.65 12.9)))


## 3. Load Landsat 8 Collection

Filter by date and bounds.

In [3]:
l8_col = og.ImageCollection("landsat-c2-l2") \
    .filterDate("2023-01-01", "2023-06-30") \
    .filterBounds(roi) \
    .filter(query={"platform": {"eq": "landsat-8"}}) \
    .select(["red", "nir08"])

print(f"Images found: {l8_col.size()}")

Images found: 11


## 4. Compute Median Composite and NDVI

Create a median composite and then calculate NDVI using `normalizedDifference`.

In [4]:
# Median composite (reprojecting for analysis)
image = l8_col.median(epsg=32643)

# Calculate NDVI: (NIR - Red) / (NIR + Red)
ndvi = image.normalizedDifference(["nir08", "red"]).rename("NDVI")

print("NDVI Image:", ndvi)

NDVI Image: og.Image({
  "type": "Image",
  "bands": [],
  "dtype": "float64",
  "shape": [
    367,
    360
  ],
  "crs": "EPSG:32643"
})


## 5. Visualize on Map

Instantiate the `Map` object and add the NDVI layer using `leafmap` backend.

In [5]:
# Create the Map instance (GEE style)
Map = og.Map()

vis_params = {
    'min': 0,
    'max': 0.8,
    'palette': 'red, yellow, green'
}

# Add the NDVI layer to the map
Map.addLayer(ndvi, vis_params, "Landsat 8 NDVI")

# Center the map on the ROI
Map.centerObject(roi, zoom=12)

# Display the Map
Map

Map(center=[12.9502105, 77.59984349999999], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_i…

## 6. Zonal Statistics

Calculate the mean NDVI for the ROI.

In [6]:
stats = ndvi.reduceRegion(
    reducer='mean', 
    geometry=roi, 
    scale=100
)
print("Mean NDVI:", stats)

Clip failed: No data found in bounds. Data variable: stackstac-13e394981317554a4e884d2451601923


Mean NDVI: {'constant': 0.3492518964350593}
